In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


# **import data**

In [3]:
data = pd.read_csv('../input/sentiment140/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',header=None)
#data = data.sample(n=10000, random_state=42) 
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
data.drop([1,2,3,4],axis=1,inplace=True)
data.columns=['target','text']
data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
X = data.iloc[:,[1]]
Y = data.iloc[:,0]
Y[Y == 4] = 1

/tmp/ipykernel_34/3957423797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[Y == 4] = 1


# **preprocessing**

In [6]:
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# # Text-preprocessing

# # # Missing Values
# # num_missing_desc = data.isnull().sum()[2]    # No. of values with msising descriptions
# # print('Number of missing values: ' + str(num_missing_desc))
# # data = data.dropna()

TAG_CLEANING_RE = "@\S+"
# Remove @tags
X['text'] = X['text'].map(lambda x: re.sub(TAG_CLEANING_RE, ' ', x))

# Smart lowercase
X['text'] = X['text'].map(lambda x: x.lower())

# Remove numbers
X['text'] = X['text'].map(lambda x: re.sub(r'\d+', ' ', x))

# Remove links
TEXT_CLEANING_RE = "https?:\S+|http?:\S|[^A-Za-z0-9]+"
X['text'] = X['text'].map(lambda x: re.sub(TEXT_CLEANING_RE, ' ', x))

# Remove Punctuation
X['text']  = X['text'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))

# Remove white spaces
X['text'] = X['text'].map(lambda x: x.strip())

# # Tokenize into words
# X['text'] = X['text'].map(lambda x: word_tokenize(x))
 
# # Remove non alphabetic tokens
# X['text'] = X['text'].map(lambda x: [word for word in x if word.isalpha()])

# # Filter out stop words
# stop_words = set(stopwords.words('english'))
# X['text'] = X['text'].map(lambda x: [w for w in x if not w in stop_words])

/tmp/ipykernel_34/2712050328.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['text'] = X['text'].map(lambda x: re.sub(TAG_CLEANING_RE, ' ', x))
/tmp/ipykernel_34/2712050328.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['text'] = X['text'].map(lambda x: x.lower())
/tmp/ipykernel_34/2712050328.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [7]:
X.head()

,text
0,y zl awww that s a bummer you shoulda got davi...
1,is upset that he can t update his facebook by ...
2,i dived many times for the ball managed to sav...
3,my whole body feels itchy and like its on fire
4,no it s not behaving at all i m mad why am i h...


# **split the data**

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("TRAIN size:", len(X_train))
print("TEST size:", len(X_test))

TRAIN size: 1280000
TEST size: 320000


In [9]:
from keras.utils import to_categorical


y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [10]:
y_test.shape

(320000, 2)

# **apply word2Vec**

In [11]:
documents = [_text.split() for _text in X_train.text] 

In [12]:
import gensim

W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)

In [13]:
w2v_model.build_vocab(documents)

In [14]:
words = list(w2v_model.wv.key_to_index.keys())
vocab_size = len(words)
print("Vocabulary size:", vocab_size)

Vocabulary size: 29578


In [15]:
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

h


(401052679, 533358528)

In [16]:
X

,text
0,y zl awww that s a bummer you shoulda got davi...
1,is upset that he can t update his facebook by ...
2,i dived many times for the ball managed to sav...
3,my whole body feels itchy and like its on fire
4,no it s not behaving at all i m mad why am i h...
...,...
1599995,just woke up having no school is the best feel...
1599996,thewdb com very cool to hear old walt intervie...
1599997,are you ready for your mojo makeover ask me fo...
1599998,happy th birthday to my boo of alll time tupac...


# **Tokenizer**

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train.text)

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

Total words 245208


In [18]:
SEQUENCE_LENGTH = 300

x_train = pad_sequences(tokenizer.texts_to_sequences(X_train.text), maxlen=SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(X_test.text), maxlen=SEQUENCE_LENGTH)

In [19]:
embedding_matrix = np.zeros((vocab_size, W2V_SIZE))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(245208, 300)


# **CNN model**

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense

# Define CNN model
model = Sequential()

embedding_layer = Embedding(input_dim=vocab_size, output_dim=W2V_SIZE, input_length=SEQUENCE_LENGTH, trainable=False)

embedding_layer.build((None,))
embedding_layer.set_weights([embedding_matrix])

model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Adjust num_classes based on your classification task

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy*100)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │    73,562,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,562,400 (280.62 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 73,562,400 (280.62 MB)

Epoch 1/10
   24/20000 ━━━━━━━━━━━━━━━━━━━━ 2:24 7ms/step - accuracy: 0.5383 - loss: 0.8388

I0000 00:00:1716145469.298432     402 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1716145469.315227     402 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


20000/20000 ━━━━━━━━━━━━━━━━━━━━ 149s 7ms/step - accuracy: 0.7905 - loss: 0.4458
Epoch 2/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 148s 7ms/step - accuracy: 0.8176 - loss: 0.4000
Epoch 3/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 148s 7ms/step - accuracy: 0.8254 - loss: 0.3859
Epoch 4/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 148s 7ms/step - accuracy: 0.8313 - loss: 0.3747
Epoch 5/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 148s 7ms/step - accuracy: 0.8348 - loss: 0.3664
Epoch 6/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 148s 7ms/step - accuracy: 0.8384 - loss: 0.3587
Epoch 7/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 149s 7ms/step - accuracy: 0.8424 - loss: 0.3502
Epoch 8/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 148s 7ms/step - accuracy: 0.8449 - loss: 0.3445
Epoch 9/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 149s 7ms/step - accuracy: 0.8469 - loss: 0.3391
Epoch 10/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 148s 7ms/step - accuracy: 0.8503 - loss: 0.3323
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.8175 - loss: 0.4131
Test Accur

**test the model with 10 random samples from testset**

In [43]:
indx = np.random.choice(len(x_test), 10, replace = False)

x_test_samples = x_test[indx]
y_test_samples = y_test[indx]

predictions = model.predict(x_test_samples)

for i in range(10):
    y_act = np.argmax(y_test_samples[i])
    y_pred = np.argmax(predictions[i])
    print(f"Sample {i+1}:")
    print(f"  Actual Label: {y_act}", f"  Predicted Label: {y_pred}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Sample 1:
  Actual Label: 1   Predicted Label: 1
Sample 2:
  Actual Label: 0   Predicted Label: 0
Sample 3:
  Actual Label: 0   Predicted Label: 0
Sample 4:
  Actual Label: 0   Predicted Label: 0
Sample 5:
  Actual Label: 1   Predicted Label: 1
Sample 6:
  Actual Label: 1   Predicted Label: 0
Sample 7:
  Actual Label: 1   Predicted Label: 1
Sample 8:
  Actual Label: 0   Predicted Label: 0
Sample 9:
  Actual Label: 0   Predicted Label: 1
Sample 10:
  Actual Label: 1   Predicted Label: 1


# **another CNN model**

In [44]:
from tensorflow.keras.layers import Dropout, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense

model = Sequential()

embedding_layer = Embedding(input_dim=vocab_size, output_dim=W2V_SIZE, input_length=SEQUENCE_LENGTH, trainable=False)

embedding_layer.build((None,))
embedding_layer.set_weights([embedding_matrix])

model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

# GlobalMaxPooling layer
model.add(GlobalMaxPooling1D())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Adjust num_classes based on your classification task

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy*100)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │    73,562,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,562,400 (280.62 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 73,562,400 (280.62 MB)

Epoch 1/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 134s 6ms/step - accuracy: 0.7420 - loss: 0.5225
Epoch 2/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 125s 6ms/step - accuracy: 0.7700 - loss: 0.4834
Epoch 3/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 124s 6ms/step - accuracy: 0.7769 - loss: 0.4725
Epoch 4/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 124s 6ms/step - accuracy: 0.7808 - loss: 0.4656
Epoch 5/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 124s 6ms/step - accuracy: 0.7836 - loss: 0.4610
Epoch 6/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 124s 6ms/step - accuracy: 0.7862 - loss: 0.4562
Epoch 7/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 124s 6ms/step - accuracy: 0.7883 - loss: 0.4527
Epoch 8/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 124s 6ms/step - accuracy: 0.7904 - loss: 0.4496
Epoch 9/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 124s 6ms/step - accuracy: 0.7920 - loss: 0.4465
Epoch 10/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 124s 6ms/step - accuracy: 0.7934 - loss: 0.4440
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.7752 - loss: 0.4801

# **third model**

In [20]:
from tensorflow.keras.layers import Dropout, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization

model = Sequential()

embedding_layer = Embedding(input_dim=vocab_size, output_dim=W2V_SIZE, input_length=SEQUENCE_LENGTH, trainable=False)

embedding_layer.build((None,))
embedding_layer.set_weights([embedding_matrix])

model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())

# Dense layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Adjust num_classes based on your classification task

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy*100)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │    73,562,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,562,400 (280.62 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 73,562,400 (280.62 MB)

Epoch 1/10
   12/20000 ━━━━━━━━━━━━━━━━━━━━ 5:15 16ms/step - accuracy: 0.5106 - loss: 0.9776 

I0000 00:00:1716191575.564712     398 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20000/20000 ━━━━━━━━━━━━━━━━━━━━ 176s 8ms/step - accuracy: 0.7786 - loss: 0.4717
Epoch 2/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 165s 8ms/step - accuracy: 0.8171 - loss: 0.4072
Epoch 3/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 165s 8ms/step - accuracy: 0.8322 - loss: 0.3786
Epoch 5/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 164s 8ms/step - accuracy: 0.8367 - loss: 0.3690
Epoch 6/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 165s 8ms/step - accuracy: 0.8408 - loss: 0.3614
Epoch 7/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 165s 8ms/step - accuracy: 0.8444 - loss: 0.3529
Epoch 8/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 164s 8ms/step - accuracy: 0.8484 - loss: 0.3469
Epoch 9/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 165s 8ms/step - accuracy: 0.8518 - loss: 0.3393
Epoch 10/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 165s 8ms/step - accuracy: 0.8547 - loss: 0.3326
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.8229 - loss: 0.3917
Test Accuracy: 82.26156234741211
